# Natural-Language-Processing-for-predicting-hospital-readmission( Azure)
> This note book is intended to run on an azure HD insights cluster

The following files must have been constructed(pre-Azure ) and uploaded to your HDINSIGHTS storage container

```python
'notes_discharge_pd.csv' , 'ADMISSIONS.CSV'  and 'IDC9_filter.csv'
```

In [1]:
%%info

In [1]:
%%configure -f 
{"name":"remotesparkmagics-sample", "executorMemory": "28G", "executorCores":6,"driverMemory":"28g", "heartbeatTimeoutInSecond":6000}

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
31,application_1541818620294_0035,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=remotesparkmagics-sample>

In [4]:
import pandas as pd 
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions  as psf
from pyspark.sql import Window 
from pyspark.sql.types  import DateType
#import pandas as pd
#pd.set_option('display.max_colwidth', -1)
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType

In [5]:
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf

In [5]:
# wasb[s]://cse6250-2018-11-10t02-36-55-285z@flufy_lobster69.blob.core.windows.net/<path>

## Import data

In [ ]:
# Csv containing more IDC diagnosis and procedure codes 
IDC9_filter=(spark.read
    .option("inferschema", "true")
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .csv("wasb:///IDC9_filter.csv"))

In [5]:
admissions_sdf=(spark.read
    .option("inferschema", "true")
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .csv("wasb:///ADMISSIONS.csv"))

In [46]:
# note_struct = StructType([StructField("ROW_ID", IntegerType(), True),
#                       StructField("SUBJECT_ID", IntegerType(), True),
#                       StructField("HADM_ID", StringType(), True),
#                       StructField("CHARTDATE", StringType(), True),
#                       StructField("CHARTTIME", StringType(), True),
#                       StructField("STORETIME", StringType(), True),
#                       StructField("CATEGORY", StringType(), True),
#                       StructField("DESCRIPTION", StringType(), True),
#                       StructField("CGID", StringType(), True),
#                       StructField("ISERROR", StringType(), True),
#                       StructField("TEXT", StringType(), True)])
notes_sdf=(spark.read
    .option("inferschema", "true")
    .option("header", "true")
    .option("delimiter", "|")
    .csv("wasb:///notes_discharge_pd.csv"))  

In [47]:
admissions_sdf.cache().take(1)

[Row(ROW_ID=21, SUBJECT_ID=22, HADM_ID=165315, ADMITTIME=datetime.datetime(2196, 4, 9, 12, 26), DISCHTIME=datetime.datetime(2196, 4, 10, 15, 54), DEATHTIME=None, ADMISSION_TYPE='EMERGENCY', ADMISSION_LOCATION='EMERGENCY ROOM ADMIT', DISCHARGE_LOCATION='DISC-TRAN CANCER/CHLDRN H', INSURANCE='Private', LANGUAGE=None, RELIGION='UNOBTAINABLE', MARITAL_STATUS='MARRIED', ETHNICITY='WHITE', EDREGTIME=datetime.datetime(2196, 4, 9, 10, 6), EDOUTTIME=datetime.datetime(2196, 4, 9, 13, 24), DIAGNOSIS='BENZODIAZEPINE OVERDOSE', HOSPITAL_EXPIRE_FLAG=0, HAS_CHARTEVENTS_DATA=1)]

In [9]:

#import notes 
# notes_df = spark.read.csv("wasb:///NOTEEVENTS.csv", header=True,schema=note_struct)
#csvFile = spark.read.csv('wasb:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv', header=True, inferSchema=True)

In [48]:
notes_sdf.cache().take(1)

[Row(SUBJECT_ID=22532, HADM_ID=167853.0, CATEGORY='Discharge summary', TEXT='Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]   Service: ADDENDUM:  RADIOLOGIC STUDIES:  Radiologic studies also included a chest CT, which confirmed cavitary lesions in the left lung apex consistent with infectious process/tuberculosis.  This also moderate-sized left pleural effusion.  HEAD CT:  Head CT showed no intracranial hemorrhage or mass effect, but old infarction consistent with past medical history.  ABDOMINAL CT:  Abdominal CT showed lesions of T10 and sacrum most likely secondary to osteoporosis. These can be followed by repeat imaging as an outpatient.                                [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]  Dictated By:[**Hospital 1807**] MEDQUIST36  D:  [**2151-8-5**]  12:11 T:  [**2151-8-5**]  12:21 JOB#:  [**Job Number 1808**] ')]

In [11]:
notes_sdf.dtypes, admissions_sdf.dtypes

([('SUBJECT_ID', 'int'), ('HADM_ID', 'double'), ('CATEGORY', 'string'), ('TEXT', 'string')], [('ROW_ID', 'int'), ('SUBJECT_ID', 'int'), ('HADM_ID', 'int'), ('ADMITTIME', 'timestamp'), ('DISCHTIME', 'timestamp'), ('DEATHTIME', 'timestamp'), ('ADMISSION_TYPE', 'string'), ('ADMISSION_LOCATION', 'string'), ('DISCHARGE_LOCATION', 'string'), ('INSURANCE', 'string'), ('LANGUAGE', 'string'), ('RELIGION', 'string'), ('MARITAL_STATUS', 'string'), ('ETHNICITY', 'string'), ('EDREGTIME', 'timestamp'), ('EDOUTTIME', 'timestamp'), ('DIAGNOSIS', 'string'), ('HOSPITAL_EXPIRE_FLAG', 'int'), ('HAS_CHARTEVENTS_DATA', 'int')])

## subset to used data

In [49]:
#create list of used columns 
adm_cols=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',  'ADMISSION_TYPE']
notes_cols=['SUBJECT_ID', 'HADM_ID', 'CATEGORY','TEXT' ]
## subset notes and admissions to relavant data
adm_used= admissions_sdf[adm_cols]



notes_used = notes_sdf[notes_cols]
# change key type 
notes_used = notes_used.select('SUBJECT_ID', psf.col('HADM_ID').cast('int').alias('HADM_ID'), 'CATEGORY','TEXT' ).cache()
adm_used.printSchema()
notes_used.printSchema()

root
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ADMITTIME: timestamp (nullable = true)
 |-- DISCHTIME: timestamp (nullable = true)
 |-- DEATHTIME: timestamp (nullable = true)
 |-- ADMISSION_TYPE: string (nullable = true)

root
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- TEXT: string (nullable = true)

## Prepare admissions

In [50]:
sort_adm=adm_used.orderBy( psf.col("SUBJECT_ID"), psf.col('ADMITTIME'))
w = Window.partitionBy("SUBJECT_ID").orderBy(psf.desc('ADMITTIME'))
#next_adm= sort_adm.withColumn('next_admit', psf.lag(col('ADMITTIME')).over(w))
next_adm=sort_adm.select('*', psf.lag(psf.col('ADMITTIME')).over(w).alias('NEXT_ADMISSION')
                , psf.lag(psf.col('ADMISSION_TYPE')).over(w).alias('NEXT_ADMISSION_TYPE')).cache()

In [51]:
#example
next_adm\
.select('SUBJECT_ID','HADM_ID' ,'ADMITTIME' ,'NEXT_ADMISSION_TYPE','ADMISSION_TYPE','NEXT_ADMISSION')\
.where(psf.col("NEXT_ADMISSION").isNotNull())\
.show(2)

+----------+-------+-------------------+-------------------+--------------+-------------------+
|SUBJECT_ID|HADM_ID|          ADMITTIME|NEXT_ADMISSION_TYPE|ADMISSION_TYPE|     NEXT_ADMISSION|
+----------+-------+-------------------+-------------------+--------------+-------------------+
|      4101| 161951|2103-01-22 18:01:00|           ELECTIVE|     EMERGENCY|2106-09-17 11:45:00|
|      4900| 117413|2203-06-29 08:00:00|           ELECTIVE|      ELECTIVE|2203-10-26 13:15:00|
+----------+-------+-------------------+-------------------+--------------+-------------------+
only showing top 2 rows

### Add days to readmission

In [52]:
adm_datediff= next_adm.withColumn("days_next_admit", psf.datediff("NEXT_ADMISSION", "DISCHTIME")).cache()

In [53]:
##example :) 

adm_datediff.take(1)[0]['SUBJECT_ID']

148

## Prepare notes 

In [54]:
notes_used.select("CATEGORY").distinct().show()

+-----------------+
|         CATEGORY|
+-----------------+
|Discharge summary|
+-----------------+

In [55]:
## filter to discharge
#notes_dis_sum = notes_used.filter( notes_used.CATEGORY == 'Discharge summary')
## create index to dedup
w =  Window.partitionBy("SUBJECT_ID",'HADM_ID').orderBy(psf.desc('HADM_ID'))
notes_dis_sum  = notes_used.withColumn("row_number", psf.row_number().over(w)).cache()

In [56]:
## dedup
notes_dis_dedup=notes_dis_sum.filter(notes_dis_sum.row_number==1).cache()

In [57]:
notes_dis_sum.groupBy("SUBJECT_ID",'HADM_ID').count().filter(psf.col('count')>1).show(3)
notes_dis_dedup.groupBy("SUBJECT_ID",'HADM_ID').count().filter(psf.col('count')>1).show(3)


+----------+-------+-----+
|SUBJECT_ID|HADM_ID|count|
+----------+-------+-----+
|      1453| 122894|    2|
|      1616| 176986|    2|
|      2265| 100548|    2|
+----------+-------+-----+
only showing top 3 rows

+----------+-------+-----+
|SUBJECT_ID|HADM_ID|count|
+----------+-------+-----+
+----------+-------+-----+

In [58]:
# join notes and admisions remove newborns 
note_admit_join=adm_datediff\
.filter(psf.col('ADMISSION_TYPE')!='NEWBORN').join(notes_dis_dedup, ['HADM_ID','SUBJECT_ID'],'left').cache()

In [59]:
### check to see if cutoff of 30 is reasonable
sum_between= adm_datediff.select('days_next_admit').groupBy().sum().take(1)[0]['sum(days_next_admit)']
count_between= adm_datediff.select('days_next_admit').count()
sum_between/count_between

85.44102685838307

## create label 

In [60]:
#create label
note_admit_label=note_admit_join.select('*', psf.when(psf.col('days_next_admit')<30,1).otherwise(0).alias('label')).cache()

In [6]:

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [23]:
# import string
# def tokenizer_better(text):
#     # tokenize the text by replacing punctuation and numbers with spaces and lowercase all words
#     if text != None:
#         punc_list = string.punctuation+'0123456789'
#         t = str.maketrans(dict.fromkeys(punc_list, " "))
#         text = text.lower().translate(t)
#         tokens = word_tokenize(text)
#     else:
#         tokens = []
#     return tokens
#spark_tokenizer = psf.udf(lambda text: tokenizer_better(text), ArrayType(StringType()))

## clean text

In [62]:
## to clean
import string
def text_cleaner(text):
    # tokenize the text by replacing punctuation and numbers with spaces and lowercase all words
    if text != None:
        punc_list = string.punctuation+'0123456789'
        t = str.maketrans(dict.fromkeys(punc_list, " "))
        text = text.lower().translate(t)
        #tokens = word_tokenize(text)
    else:
        text = ''
    return text
#register UDF

spark_cleaner= psf.udf(lambda text: text_cleaner(text),StringType())

# stopwords 

In [63]:
alphabet_low = []
for letter in range(97,123):
    alphabet_low.append(chr(letter))
stopwords=['','it', 'did', 'he', 'mustn', 'whom', "that'll", 'why', 'ain', "it's", 'isn', 'off', 'only', 'ourselves', 'which', 'to', 'she', 'during', "wouldn't", 'have', 'if', 'against', 'some', 'for', 'i', "don't", "didn't", 'ours', 'itself', 'should', 'very', 'those', 'him', 'just', 'll', "mustn't", 'out', 'theirs', 'each', 'needn', 'd', 'herself', 'few', 'himself', 'these', 'couldn', 'or', 'not', 'when', 'with', 'does', "needn't", "weren't", 'into', 'm', 'y', 'we', "wasn't", 'both', "doesn't", 'you', 'by', 'didn', 'our', 'was', 'wouldn', "haven't", 'more', 'who', 'same', 't', 'a', 'hadn', 'what', 'won', "couldn't", 'nor', 'they', 'shouldn', 'this', 'there', 'an', 'where', 'while', 'shan', 'has', 'yourself', 'so', 'had', 'myself', 'do', 'been', 'all', 'his', 'haven', 'between', "you've", 'doesn', 'at', 'me', 'then', 'until', 'above', 'ma', 'further', 'and', 'of', 're', 'o', 'on', 'up', 'its', 'her', 've', 'because', "isn't", 'than', "shan't", 'are', 'now', 'is', 'yours', 'will', 'be', 'mightn', 'their', 'being', 'about', 'your', "won't", 'don', "should've", 'wasn', 'how', 'am', 'weren', 'were', "hasn't", 'under', 'as', 'can', "you'd", 'from', 'once', "you'll", 'doing', 'too', 'down', "mightn't", 'before', 'themselves', "aren't", 'having', 'aren', 'no', 'over', 'them', 'through', 'but', "shouldn't", 'after', 's', 'hers', 'below', 'in', 'again', 'most', 'own', 'hasn', 'my', 'the', 'that', 'other', 'yourselves', 'such', 'any', "she's", 'here', "you're", "hadn't"]

In [ ]:
idc9_word_list=list(IDC9_filter.toPandas().IDC_filter)

In [72]:
stop_list=alphabet_low+stopwords

In [73]:
### define python -> spark sql function to remove stop words and find description words
def stopword_remove(word_list):
    cleaned=[w for w in word_list if not w in stop_list if len(w)>2]
    code_descriptions=[w for w in cleaned if  w in idc9_word_list ]
    return code_descriptions
spark_stop_remove = psf.udf(lambda text: stopword_remove(text), ArrayType(StringType()))   


## clean to just word 

In [74]:
note_admit_clean=note_admit_label.select('*',spark_cleaner(psf.col("TEXT")).alias("TEXT_clean")).cache()
#.limit(1).take(1)[0]['<lambda>(TEXT)']

In [75]:
tokenizer = Tokenizer(inputCol="TEXT_clean", outputCol="tokens")
note_admit_token = tokenizer.transform(note_admit_clean).cache()

In [76]:
# run to cache
note_admit_token.count()

51113

In [77]:
# remove stops 
note_admit_stop = note_admit_token.select('*',spark_stop_remove(psf.col("tokens")).alias("tokens_clean")).cache()

In [78]:
note_admit_stop.select('tokens_clean').take(1)

[Row(tokens_clean=['date', 'discharge', 'date', 'date', 'birth', 'sex', 'medicine', 'patient', 'drugs', 'complaint', 'nausea', 'major', 'surgical', 'procedure', 'central', 'history', 'present', 'illness', 'old', 'ischemic', 'severe', 'type', 'nausea', 'hypotension', 'heart', 'failure', 'clinic', 'kidneys', 'severe', 'renal', 'artery', 'orthopnea', 'vomiting', 'diarrhea', 'recent', 'use', 'decreased', 'initial', 'insulin', 'blood', 'urine', 'central', 'past', 'medical', 'history', 'ischemic', 'severe', 'type', 'blindness', 'complete', 'occlusion', 'cri', 'renal', 'social', 'history', 'alone', 'smoking', 'alcohol', 'use', 'family', 'history', 'non', 'physical', 'exam', 'grade', 'sem', 'diastolic', 'around', 'iliac', 'fossa', 'evidence', 'abscess', 'right', 'lung', 'base', 'pneumonia', 'atelectasis', 'clinical', 'bilateral', 'pleural', 'right', 'left', 'stomach', 'gastroparesis', 'vascular', 'transplant', 'kidney', 'right', 'lower', 'brief', 'course', 'old', 'type', 'history', 'vessel', '

In [79]:
note_admit_stop.columns

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'TEXT', 'row_number', 'label', 'TEXT_clean', 'tokens', 'tokens_clean']

In [82]:
#remove lists and columns not fed to create features
note_admit_stop_reduce=note_admit_stop.select(note_admit_stop.columns[:10]+['tokens_clean','label']).cache()

In [ ]:
note_admit_stop_reduce.count()

In [ ]:
note_admit_stop.write.parquet('wasb:///final_idc9_filt.parquet')

In [ ]:
## Sometimes Azure lags here A quick fix was writing then reading

In [6]:
final_idc9_filt = spark.read.parquet('wasb:///final_idc9_filt.parquet')


In [7]:
#write parquet this is the base 
final_idc9_filt.repartition(numPartitions=1).write.parquet('wasb:///final_tokens_with_text.parquet')

In [7]:
final_idc9_filt.columns

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'TEXT', 'row_number', 'label', 'TEXT_clean', 'tokens', 'tokens_clean']

In [8]:
# if from scratch use 'note_admit_stop'
final_data=final_idc9_filt.select(final_idc9_filt.columns[:10]+['tokens_clean','label']).cache()

In [10]:
#keep
# final_data.repartition(numPartitions=1).write.parquet('wasb:///final_data_V2.parquet')